Given two arrays $A[N]$ and $B[N]$, return the number of $Z[N]$, such that $Z[0] \le Z[1] \le \dots \le Z[n-1]$ and $A[i] \le Z[i] < B[i]$.

Polynomi

In [245]:
def solve_w(y, N):
    n = len(y)
    w = [0 for i in range(max(n,N))]
    g = [[0 for j in range(n+1)] for i in range(n)]
    for i in range(n):
        g[i][0] = 1
        g[i][n] = y[i]
        for j in range(1, n):
            g[i][j] = g[i][j-1] * (i+1)
    for i in range(n):
        if g[i][i] == 0:
            for k in range(i+1, n):
                if g[k][i] != 0:
                    g[i], g[k] = g[k], g[i]
                    break
        if g[i][i] != 1:
            s = g[i][i]
            for j in range(n+1):
                g[i][j] /= s
        for l in range(0, n):
            if l == i:
                continue
            s = g[l][i]
            for j in range(i, n+1):
                g[l][j] -= s * g[i][j]
    for i in range(n):
        w[i] = g[i][n]
    return w

def sumN(w, x):
    n = len(w)
    ret = 0
    t = 1
    for i in range(n):
        ret += w[i] * t
        t *= x
    return ret

def sumX2Y(w, x, y):
    return sumN(w, y-1) - sumN(w, x-1)

def get_constant(x, n):
    ret =[0 for i in range(n)]
    ret[0] = x
    return ret

def add_vec(x, y):
    return [x[i] + y[i] for i in range(len(x))]

def sub_vec(x, y):
    return [x[i] - y[i] for i in range(len(x))]

def times_vec(x, t):
    return [x[i] * t  for i in range(len(x))]

# w = solve_w([0, 1, 3], 0)
# print(w)
# for i in range(1,10):
#     print(sumN(w, i))

In [292]:
class PowerSum:
    def __init__(self, N):
        self.N = N
        self.W = []
        for i in range(N):
            s = [0]
            for j in range(1, i+2):
                s.append(s[j-1] + ((j)**i))
            self.W.append(solve_w(s, N+1))
            
    def sum(self, n, k):
        if n == None:
            return self.W[k].copy()
        return self.get_constant(sumN(self.W[k], n))
    
    def sumPoly(self, w, n):
        ret = self.get_constant(0)
        for i in range(len(w)):
            if w[i] == 0:
                continue
            cur = self.sum(n, i)
            cur = times_vec(cur, w[i])
            ret = add_vec(ret, cur)
        return ret
    def get_constant(self, x):
        ret =[0 for i in range(self.N+1)]
        ret[0] = x
        return ret
    
        
ps = PowerSum(10)
print("sum 1..9", ps.sum(10,1)[0])

# An =  1 + 2n
# A1 = 3
# A2 = 5
# A3 = 7
print("sum A1..A3", ps.sumPoly([1,2],4)[0])

# An = 3 - n^2
# A1 = 2
# A2 = -1
# A3 = -6
print(ps.sumPoly([3, 0, -1, 0], 2)[0])
print(ps.sumPoly([3, 0, -1, 0], 3)[0])
print(ps.sumPoly([3, 0, -1, 0], 4)[0])

sum 1..9 45.0
sum A1..A3 15.0
2.0000000000000004
1.0
-4.999999999999998


In [294]:
def calc_l_r(ps, prev, left, right):
    lsum = ps.sumPoly(prev, left)
    rsum = ps.sumPoly(prev, right)
    return sub_vec(rsum, lsum)

def solve_azb(a, b):
    n = len(a)
    ps = PowerSum(n+1)
    s = sorted(list(set(a + b)))
    ind = {}
    for i in range(len(s)):
        ind[s[i]] = i
        
    prev_left = ind[a[n-1]]
    prev_right = ind[b[n-1]]
    prev = [ps.get_constant(0) for i in range(len(s))]
    for i in range(prev_left, prev_right):
        prev[i] = ps.get_constant(1)
    for i in range(n-2,-1,-1):
        left = ind[a[i]]
        right = ind[b[i]]
        cur = [ps.get_constant(0) for i in range(len(s))]
        for x in range(left, right):
            for y in range(max(x,prev_left), prev_right):
                if x == y:
                    cur[x] = add_vec(cur[x], calc_l_r(ps, prev[y], None, s[y+1]))
                else:
                    cur[x] = add_vec(cur[x], calc_l_r(ps, prev[y], s[y], s[y+1]))
        prev = cur
        prev_left = left
        prev_right = right
        
    ans = 0
    for x in range(left, right):
        z = calc_l_r(ps, cur[x], s[x], s[x+1])
        ans += z[0]
    return ans
    
# a = [1, 2], b = [3, 4]
# (1, 2), (1, 3), (2, 2), (2, 3)  
# solve_azb([1,2], [3,4])

a = [2, 3]
b = [5, 5]
# 2,3 3,3


print("solve =", solve_azb(a, b))

ps = PowerSum(3)
calc_l_r(ps, [1, 0, 0, 0], None, 3)






solve = 5.0


[3, -1, 0, 0]

In [284]:
def helper(a, b, c):
    if len(a) == 0:
        return 1
    ret = 0
    for i in range(max(a[0], c), b[0]):
        ret += helper(a[1:], b[1:], i)
    return ret
def simple_solve_azb(a, b):
    return helper(a, b, 1)

a = [1, 6, 9, 15]
b = [10, 20, 20, 25]

print(solve_azb(a, b), simple_solve_azb(a, b))



7140.0 7140
